In [75]:
import pandas as pd
import numpy as np
import os
import tqdm
import time
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

from google.colab import drive
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import LeaveOneOut, train_test_split

from datetime import datetime

 For GPU

In [35]:
drive.mount('/content/drive')

study_fold = '/content/drive/MyDrive/Thesis:Epilepsy/study/'
dataset_path = os.path.join(study_fold, "stats/datasetRadiomicsReduced.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [260]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [261]:
df = pd.read_csv(dataset_path, index_col="ID")

KeyboardInterrupt: ignored

In [262]:
class DatasetDiffusionMRI(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    features = self.X.iloc[idx, :].to_numpy()
    label = self.y.iloc[idx].to_numpy(dtype="float64")

    return features, label

In [263]:
def looPytorch():
  avg_score = []

  for train_val_idx, test_idx in LeaveOneOut().split(df):
    train_val = df.iloc[train_val_idx, :]
    test = df.iloc[test_idx, :]
    for train_idx, val_idx in LeaveOneOut().split(train_val):
      train = train_val.iloc[train_idx, :]
      val = train_val.iloc[val_idx, :]

      # train the model

      # evaluate the model, fo the hyperParameters selection

    # Select the best model and use it for the test
    # test the model, compute the score and add tot the list

  # mean of avg_score
  return np.mean(avg_score)


In [264]:
X, y, y3 = df.iloc[:, 2:], df.iloc[:,:1], df.iloc[:, 1:2]
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.12, shuffle=True, stratify=y3)
y3_train_val = y3.loc[y_train_val.index]
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.14, shuffle=True, stratify=y3_train_val)

In [323]:
class ModelTrainer():
    def __init__(self, train_dataset, val_dataset, model, device, loss_fn, optimizer, batch_size):
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.model = model.to(device)
        self.device = device
        self.loss_fn = loss_fn
        self.optimizer = optimizer

        self.train_loader = DataLoader(
            self.train_dataset,
            batch_size,
            shuffle=True
        )

        self.val_loader = DataLoader(
            self.val_dataset,
            batch_size,
            shuffle=True
        )

        self.epoch_number = 0

    def __trainOneEpoch(self, epoch_index, tb_writer):
        running_loss = 0

        for i, data in enumerate(self.train_loader):

            # Step 1: Get the features and the label
            features, labels = data
            features = features.to(self.device)
            labels = labels.to(self.device)

            # Step 2: Zero the gradient
            self.optimizer.zero_grad()

            # Step 3: Make predictions
            outputs = self.model(features)

            # Step 4: Compute the loss
            loss = self.loss_fn(outputs, labels)

            # - : Compute the gradient
            loss.backward()

            # Step 5: Adjust weights
            self.optimizer.step()

            # Data and report Statistics
            curr_loss = loss.item()
            running_loss += curr_loss

            # since the small dataset, report the loss at each batch
            print("\tbatch {} loss: {}".format(i+1, curr_loss))
            tb_x = epoch_index*len(self.train_loader) + i + 1
            tb_writer.add_scalar("Loss/Train", curr_loss, tb_x)

        return running_loss/(i+1)


    def train(self, epochs):
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))

        EPOCHS = epochs

        best_vloss = 1_000_000

        for epoch in range(EPOCHS):
            print("EPOCH {}".format(self.epoch_number + 1))

            # Set the model to training mode
            self.model.train()
            avg_loss = self.__trainOneEpoch(self.epoch_number, writer)

            # Set the model to validation mode
            running_vloss = 0.0
            self.model.eval()

            with torch.no_grad():
                for i, vdata in enumerate(self.val_loader):
                    # Get the features and the labels
                    features, labels = vdata
                    features = features.to(self.device)
                    labels = labels.to(self.device)

                    voutputs = self.model(features)

                    loss = self.loss_fn(voutputs, labels)
                    running_vloss +=loss

            avg_vloss = running_vloss/(i+1)
            print("LOSS train {} valid {}".format(avg_loss, avg_vloss))

            # Data and report Statistics
            writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    self.epoch_number + 1)
            writer.flush()

            if avg_vloss < best_vloss:
                best_vloss = avg_vloss
                model_path = "model_{}_{}".format(timestamp, self.epoch_number)
                torch.save(self.model.state_dict(), model_path)

            self.epoch_number += 1


Creating my neural network

In [324]:
class NeuralNetwork(nn.Module):
  def __init__(self, in_features: int, hidden_features: int):
    super().__init__()

    self.activation = nn.ReLU()

    self.fc1 = nn.Linear(
        in_features=in_features,
        out_features=hidden_features,
        dtype=torch.float64
    )
    self.fc2 = nn.Linear(
        in_features=hidden_features,
        out_features=1,
        dtype=torch.float64
    )
    self.sigmoid = nn.Sigmoid()

  def forward(self, X):
    X = self.activation(self.fc1(X))
    X = self.fc2(X)
    X = self.sigmoid(X)
    return X

In [ ]:
network = NeuralNetwork(X_train.shape[1], 10000).to(device)

model_trainer = ModelTrainer(
    DatasetDiffusionMRI(X_train, y_train),
    DatasetDiffusionMRI(X_val, y_val),
    network,
    device,
    nn.BCELoss(),
    optim.Adam(network.parameters(), lr=500),
    batch_size=4,
)

In [ ]:
model_trainer.train(1000)

In [ ]:
X_train.shape

In [ ]:
target_loss = 1e-4

def optimize_model():
  optimizer = torch.optim.Adam(params = model.parameters(), lr=1e-3)

  while loss > target_loss:
    optimizer.zero_grad()
    outpur = model()